In [10]:
from torch.utils.data import Subset, Dataset
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from PIL import Image

import numpy as np
import random

import matplotlib.pyplot as plt

import csv
from sklearn import svm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from sklearn.linear_model import LinearRegression

import sklearn
from sklearn import datasets

from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt

import os

  # 1. Load Data

In [ ]:
import numpy as np
import torch

subject = np.load('./run_experiments_ssl_signal/subject_sample.npy')
label = np.load('./run_experiments_ssl_signal/label_sample.npy')
new_label = np.load('./run_experiments_ssl_signal/new_label_sample.npy')
eeg_data = np.load('./run_experiments_ssl_signal/eeg_data_sample.npy')
emg_data = np.load('./run_experiments_ssl_signal/emg_data_sample.npy')

subject = torch.from_numpy(subject).long()
label = torch.from_numpy(label).long()
new_label = torch.from_numpy(new_label).long()
eeg_data = torch.from_numpy(eeg_data).float()
emg_data = torch.from_numpy(emg_data).float()

SAMPLE_X = eeg_data
SAMPLE_Y = emg_data
label_tensor = new_label

print(SAMPLE_X[:1].shape)
print(SAMPLE_Y[:1].shape)
print(label_tensor[:1].shape)
# Generate Data Cross Subject

train_idx = [np.where(subject == k)[0] for k in range(0, 20)]
test_idx = [np.where(subject == k)[0] for k in range(20, 25)]

train_idx = np.concatenate(train_idx)
test_idx = np.concatenate(test_idx)

train_major_label, test_major_label = new_label[train_idx], new_label[test_idx]
train_subtle_label, test_subtle_label = label[train_idx], label[test_idx]
train_subj_label, test_subj_label = subject[train_idx], subject[test_idx]

train_X, test_X = SAMPLE_X[train_idx], SAMPLE_X[test_idx]
train_Y, test_Y = SAMPLE_Y[train_idx], SAMPLE_Y[test_idx]

  # 2. Define Networks, Costs, Adaptive Filters for Gradients

In [12]:
import torch

# Define network for classification:
class ComplexClassifier(nn.Module):
    def __init__(self, dim_features=128, num_classes = 10):
        super(ComplexClassifier, self).__init__()
        self.fc1 = nn.Linear(dim_features, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, num_classes)  # CIFAR-10 has 10 classes

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)  # No activation, CrossEntropyLoss includes softmax
        return x

# Define network for temporal network: 
class NETWORK_F_MLP(nn.Module):
    def __init__(self, input_dim = 784, HIDDEN = 200, out_dim = 200, how_many_layers = 2):
        super(NETWORK_F_MLP, self).__init__()
        self.dim = out_dim
        self.many_layer = how_many_layers
        
        self.fc_list = []
        self.bn_list = []
        
        self.fc_list.append(nn.Linear(input_dim, HIDDEN, bias=True))
        self.bn_list.append(nn.BatchNorm1d(HIDDEN))

        for i in range(0, self.many_layer-1):
            self.fc_list.append(nn.Linear(HIDDEN, HIDDEN, bias=True))
            self.bn_list.append(nn.BatchNorm1d(HIDDEN))
            
        self.fc_list = nn.ModuleList(self.fc_list)
        self.bn_list = nn.ModuleList(self.bn_list)

        self.fc_final = nn.Linear(HIDDEN, out_dim, bias=True)

    def forward(self, x):
        
        x = x.reshape(x.shape[0], -1)
        
        for i in range(0, self.many_layer):
            x = self.fc_list[i](x)
            x = torch.relu(x)
            x = self.bn_list[i](x)
        
        x = self.fc_final(x)
        x = torch.sigmoid(x)
        return x


# Define network for channel network: 
class Advanced1DCNN_channel(nn.Module):
    def __init__(self, input_channel=1, num_classes=100, input_size=4000, num_channel=60):
        super(Advanced1DCNN_channel, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=11, padding=5),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=11, padding=5),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=11, padding=5),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=11, padding=5),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(256 * 15, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
        )

        self.fc2 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.fc3 = nn.Linear(512, num_classes)
        
        self.MLP = NETWORK_F_MLP(input_dim = 128*input_channel, HIDDEN = 4000, out_dim = num_classes, how_many_layers = 1)
        
    def forward(self, x):
        bs, channel = x.shape[0], x.shape[1]
        x = x.unsqueeze(2)
        x = x.flatten(0, 1)
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = torch.sigmoid(out)
        out = out.reshape(bs, channel, -1)
        out = out.flatten(-2, -1)
        out = self.MLP(out)
        return out


# Construct the trace cost:
def return_cost_trace(RFG, track_cov_estimate_final):

    RF_E = track_cov_estimate_final[:128, :128]
    RG_E = track_cov_estimate_final[128:, 128:]
    P_E = track_cov_estimate_final[:128, 128:]

    RF_EI = torch.inverse(RF_E)
    RG_EI = torch.inverse(RG_E)

    RF = RFG[:128, :128]
    RG = RFG[128:, 128:]
    P = RFG[:128, 128:]

    COST = -RF_EI@RF@RF_EI@P_E@RG_EI@P_E.T \
            + RF_EI@P@RG_EI@P_E.T \
            - RF_EI@P_E@RG_EI@RG@RG_EI@P_E.T \
            + RF_EI@P_E@RG_EI@P.T

    return -torch.trace(COST)

# Define adaptive smoothing filters for estimating ACFs and CCFs
def adaptive_estimation(v_t, beta, square_term, i):
    v_t = beta*v_t + (1-beta)*square_term.detach()
    return v_t, (v_t/(1-beta**i))

# Define the loss function (returns log determinant costs)
def MCA_LOSS_GIVEN_R(RP, track_cov, i, dim):
    cov = RP + torch.eye((RP.shape[0])).cuda()*(.000001)
    track_cov, cov_estimate = adaptive_estimation(track_cov, 0.5, cov, i)

    cov_estimate_f = cov_estimate[:dim, :dim]
    cov_f = cov[:dim, :dim]

    cov_estimate_g = cov_estimate[dim:, dim:]
    cov_g = cov[dim:, dim:]

    LOSS = (torch.linalg.inv(cov_estimate)*cov).sum() - (torch.linalg.inv(cov_estimate_f)*cov_f).sum() -(torch.linalg.inv(cov_estimate_g)*cov_g).sum()
    return track_cov, cov_estimate, LOSS

def create_folder_if_not_exists(folder_path):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        # If the folder does not exist, create it
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")

# 3. Run Experiments

In [13]:
### Change Total Iterartion to 30001 to run the full epoches. Here we run 501 iterartions for testing the codes. 
# total_iteration = 501
total_iteration = 30001 

# torch.cuda.set_device(2)

# Measuring Dependence between EEG and EMG
SAMPLE_X = eeg_data
SAMPLE_Y = emg_data

label_tensor = new_label

indices = np.arange(SAMPLE_X.shape[0])

from sklearn.model_selection import train_test_split
train_X, test_X, train_labels, test_labels, idx_train, idx_test = train_test_split(torch.cat((SAMPLE_X, SAMPLE_Y), 1), label_tensor, indices, test_size=0.2, random_state=42)
train_X, train_Y, test_X, test_Y = train_X[:, :60], train_X[:, 60:], test_X[:, :60], test_X[:, 60:]

NET_1 = Advanced1DCNN_channel(SAMPLE_X.shape[1], 128, 4000).cuda()
NET_2 = Advanced1DCNN_channel(SAMPLE_Y.shape[1], 128, 4000).cuda()

classifier = ComplexClassifier(dim_features = 128, num_classes = label_tensor.max()+1).cuda()
criterion = nn.CrossEntropyLoss()

beta1 = 0.9
beta2 = 0.999

lr1 = 0.0005
lr2 = 0.0005

optimizer_1 = optim.Adam([
    {'params': NET_1.parameters(), 'lr': lr2, 'betas': (beta1, beta2)},
], amsgrad = True)

optimizer_2 = optim.Adam([
    {'params': NET_2.parameters(), 'lr': lr2, 'betas': (beta1, beta2)},
], amsgrad = True)

optimizer_classifier = optim.Adam([
    {'params': classifier.parameters(), 'lr': lr2, 'betas': (beta1, beta2)},
], amsgrad = True)

save_curve = []
eig_list = []
classifier_error = []

track_cov_final = torch.zeros((128+128)).cuda()
track_cov_estimate_final = torch.zeros((128+128)).cuda()

save_path = './save_exp_inter_subj/'
create_folder_if_not_exists(save_path)

for i in range(1, total_iteration):
    batch_size = 100
    batch_indices = torch.randint(0, train_X.shape[0], (batch_size,))
    input_x = train_X[batch_indices]
    input_y = train_Y[batch_indices]
    
    feature_1 = NET_1(input_x.cuda())
    feature_2 = NET_2(input_y.cuda())
    
    # Calculating ACFs and CCFs
    RF = (feature_1.T@feature_1)/feature_1.shape[0]
    RG = (feature_2.T@feature_2)/feature_2.shape[0]
    P = (feature_1.T@feature_2)/feature_2.shape[0]
    
    input_dim, output_dim = RF.shape[1], RG.shape[1]
    RFG = torch.zeros((input_dim+output_dim, input_dim+output_dim)).cuda()
    RFG[:input_dim, :input_dim] = RF
    RFG[input_dim:, input_dim:] = RG
    RFG[:input_dim, input_dim:] = P
    RFG[input_dim:, :input_dim] = P.T
    
    # Track the ACFs and CCFs and construct the cost
    track_cov_final, track_cov_estimate_final, COST = MCA_LOSS_GIVEN_R(RFG, track_cov_final, i, 128)
    COST = return_cost_trace(RFG, track_cov_estimate_final)
    
    labels_x = train_labels[batch_indices]
    
    # Train classifier (calling .detach() such that no graidents are passed from dependence networks)
    output_class = classifier(feature_1.detach())
    loss = criterion(output_class, labels_x.cuda())    
    (COST+loss).backward()
    
    optimizer_1.step()  
    optimizer_2.step()  
    optimizer_classifier.step()  

    optimizer_1.zero_grad()  
    optimizer_2.zero_grad()      
    optimizer_classifier.zero_grad()  

    
    E1,V1 = torch.linalg.eigh(track_cov_estimate_final[:128, :128])
    E2,V2 = torch.linalg.eigh(track_cov_estimate_final[128:, 128:])

    RF_NORM = V1@torch.diag(E1**(-1/2))@V1.T
    RG_NORM = V2@torch.diag(E2**(-1/2))@V2.T

    P = track_cov_estimate_final[:input_dim, input_dim:]
    P_STAR = RF_NORM@P@RG_NORM    

    U, S, V = torch.svd(P_STAR)
    eig_list.append(S.detach().cpu().numpy())
    
    RF_NORM = RF_NORM.detach().cpu()
    U = U.detach().cpu()
    
    RG_NORM = RG_NORM.detach().cpu()
    V = V.detach().cpu()
    
    eig_list.append(S.detach().cpu().numpy())
    
    TSD = S.sum()
    classifier_error.append(TSD.item())

    if i%100 == 0:
        print('Iteration', i, 'Trace Cost', TSD.item(), 'Classifer Cost (Trained Separately)', loss.item())
        
        if i in [200, 1000, 5000, 30000]:

            np.save(save_path+'cost_iter{0}.npy'.format(i), classifier_error)
            torch.save(NET_1.state_dict(), save_path+"NET_1_iter{0}.pth".format(i))
            torch.save(NET_2.state_dict(), save_path+"NET_2_iter{0}.pth".format(i))

        if i%500 == 0:
            NET_1.eval()
            classifier.eval()

            val_loss = 0.0
            mse_loss = 0.0
            correct = 0
            criterion = nn.CrossEntropyLoss()

            with torch.no_grad():
                for index, k in enumerate(range(0, test_X.shape[0], 200)):

                    output_class = classifier(NET_1(test_X[k:k+200].cuda()))
                    val_loss += criterion(output_class, test_labels[k:k+200].cuda()).item()
                    pred = output_class.argmax(dim=1, keepdim=True).detach().cpu()
                    correct += pred.eq(test_labels[k:k+200].view_as(pred)).sum().item()

            print('Test Accuracy of Primary Movements:', correct/test_X.shape[0])
            
            NET_1.train()
            classifier.train()

Folder './save_exp_inter_subj/' already exists.
Iteration 100 Trace Cost 66.80906677246094 Classifer Cost (Trained Separately) 1.0191929340362549
Iteration 200 Trace Cost 67.44324493408203 Classifer Cost (Trained Separately) 0.699934184551239
Iteration 300 Trace Cost 66.84186553955078 Classifer Cost (Trained Separately) 0.7164157032966614
Iteration 400 Trace Cost 67.8227310180664 Classifer Cost (Trained Separately) 0.5167750716209412
Iteration 500 Trace Cost 68.00826263427734 Classifer Cost (Trained Separately) 0.4972356855869293
Test Accuracy of Primary Movements: 0.6960013384641124
Iteration 600 Trace Cost 67.86121368408203 Classifer Cost (Trained Separately) 0.4184565842151642
Iteration 700 Trace Cost 67.99005889892578 Classifer Cost (Trained Separately) 0.38334381580352783
Iteration 800 Trace Cost 68.77568054199219 Classifer Cost (Trained Separately) 0.3162824213504791
Iteration 900 Trace Cost 68.3990249633789 Classifer Cost (Trained Separately) 0.3273831903934479
Iteration 1000 Tr